In [23]:
!pip install jupyter_dash
!pip install --upgrade plotly
!pip install --upgrade dash

In [24]:
# To run this file, Win Start > cmd > file dir > run: python spacex_dash_app.py
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [25]:
# Preview the first 5 lines of the loaded data
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
spacex_df.to_csv('/content/drive/MyDrive/IBM/Curso10.CienciaDatosAplicadaCapstone/Modulo3.PanelControl/spacex_launch_dash.csv', index=False)

filepath ='/content/drive/MyDrive/IBM/Curso10.CienciaDatosAplicadaCapstone/Modulo3.PanelControl/spacex_launch_dash.csv'
dfprueba = pd.read_csv(filepath, header=0)
dfprueba.head(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [28]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Dropdown list(s)
launch_site_list = []
launch_site_list.append('ALL')
for index, row in spacex_df['Launch Site'].value_counts().to_frame().iterrows():
    launch_site_list.append(row.name)

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': i, 'value': i} for i in launch_site_list],
                                    style={'width':'100%', 'padding':'3px', 'font-size': '20px', 'text-align-last': 'left'},
                                    value='ALL'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=min_payload, max=max_payload, step=1000, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[['Launch Site', 'class']].groupby(by=['Launch Site'], as_index=False).mean()
        fig = px.pie(filtered_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[['Launch Site', 'class']][spacex_df['Launch Site'] == entered_site]
        mean = filtered_df.groupby(by='Launch Site', as_index=False).mean()
        means = {}
        means[1] = mean['class'][0]
        means[0] = 1 - means[1]
        fig = px.pie(values=means.values(), names=means.keys(),
                     title=f'Total Success Launches by Site: {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, payload_range):
    # print('min:', payload_range[0], '\tmax:', payload_range[1])
    # print(entered_site)
    if entered_site == 'ALL':
        payload_filtered_df = spacex_df[['Payload Mass (kg)', 'Booster Version Category', 'Launch Site', 'class']][(spacex_df['Payload Mass (kg)'] <= payload_range[1]) & (spacex_df['Payload Mass (kg)'] >= payload_range[0])]
    else:
        payload_filtered_df = spacex_df[['Payload Mass (kg)', 'Booster Version Category', 'Launch Site', 'class']][(spacex_df['Payload Mass (kg)'] <= payload_range[1]) &
                                                                                                                   (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                                                                                                   (spacex_df['Launch Site'] == entered_site)]
    fig = px.scatter(data_frame=payload_filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


# Finding Insights Visually
# Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:
#
# Which site has the largest successful launches?
### KSC LC-39A
# Which site has the highest launch success rate?
### KSC LC-39A
# Which payload range(s) has the highest launch success rate?
### 2000 - 4000
# Which payload range(s) has the lowest launch success rate?
### 6000 - 9000
# Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate

# Encontrar información visualmente
# Ahora, con el panel de control completado, debería poder usarlo para analizar los datos de lanzamiento de SpaceX
# y responder a las siguientes preguntas:
# ¿Qué sitio tiene los lanzamientos más exitosos?
### KSC LC-39A
# ¿Qué sitio tiene la tasa de éxito de lanzamiento más alta?
### KSC LC-39A
# ¿Qué rango(s) de carga útil tiene la tasa de éxito de lanzamiento más alta?
### 2000 - 4000
# ¿Qué rango(s) de carga útil tiene la tasa de éxito de lanzamiento más baja?
### 6000 - 9000
# Qué versión de F9 Booster (v1.0, v1.1, FT, B4, B5, etc.) tiene la tasa de éxito de lanzamiento más alta

<IPython.core.display.Javascript object>